In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast

import cv2

import albumentations
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score

from tqdm import tqdm
import argparse
import os, sys, yaml

sys.path.append('/workspace/siim-rsna-2021')
from src.logger import setup_logger, LOGGER
from src.meter import mAPMeter, AUCMeter, APMeter, AverageValueMeter
from src.utils import plot_sample_images
from src.segloss import SymmetricLovaszLoss


# import neptune.new as neptune
import wandb
import pydicom

import time
from contextlib import contextmanager

import timm

import warnings

target_columns = [
    "Negative for Pneumonia", "Typical Appearance", "Indeterminate Appearance", "Atypical Appearance", "is_none"
]

from sklearn.metrics import average_precision_score


@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [2]:
df_original = pd.read_csv("/workspace/data/df_train_study_level_npy640_3_w_bbox.csv")
df = df_original.groupby('image_id').first().reset_index()

In [18]:
# exp = ["exp414", "exp416", "exp419", "exp420", "exp605", "exp520", "exp551"]
exp = ["exp414"]

cv_all = []

print(exp)
for cv in range(5):
    pred_list = []
    pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
    pred_mean = np.mean(pred_list, axis=0)
    
    target = df[df.cv == cv][target_columns].values
    
    map_list = []
    print()
    print(f'==== cv {cv} ===')
    for i, col in enumerate(target_columns):
        map_ = average_precision_score(target[:, i], pred_mean[:, i])
        print(f"AP {col}: {map_}")
        map_list.append(map_)
    print(f"mAP: {np.mean(map_list)}")
    cv_all.append(np.mean(map_list))

print("==================================")
print(f"mAP: {np.mean(cv_all)}")
print("==================================")

['exp414']

==== cv 0 ===
AP Negative for Pneumonia: 0.8330549484126638
AP Typical Appearance: 0.8440504082921783
AP Indeterminate Appearance: 0.30789523101275906
AP Atypical Appearance: 0.44699620256206724
AP is_none: 0.8256885038011098
mAP: 0.6515370588161555

==== cv 1 ===
AP Negative for Pneumonia: 0.80129015113734
AP Typical Appearance: 0.864366884201436
AP Indeterminate Appearance: 0.3652096290231158
AP Atypical Appearance: 0.3129861868184094
AP is_none: 0.7931422115332701
mAP: 0.6273990125427142

==== cv 2 ===
AP Negative for Pneumonia: 0.8423495904466817
AP Typical Appearance: 0.858619564297686
AP Indeterminate Appearance: 0.3121096699648679
AP Atypical Appearance: 0.326941111635475
AP is_none: 0.8381163995171206
mAP: 0.6356272671723663

==== cv 3 ===
AP Negative for Pneumonia: 0.8138741306167023
AP Typical Appearance: 0.8458505994626176
AP Indeterminate Appearance: 0.3036723704080465
AP Atypical Appearance: 0.3849490107509795
AP is_none: 0.808985344144999
mAP: 0.63146629107666

# lb best

In [22]:
exp = ["exp414", "exp416", "exp419", "exp420", "exp605", "exp520", "exp551"]

cv_all = []

print(exp)
for cv in range(5):
    pred_list = []
    pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
    pred_mean = np.mean(pred_list, axis=0)
    
    target = df[df.cv == cv][target_columns].values
    
    map_list = []
    print()
    print(f'==== cv {cv} ===')
    for i, col in enumerate(target_columns):
        map_ = average_precision_score(target[:, i], pred_mean[:, i])
        print(f"AP {col}: {map_}")
        map_list.append(map_)
    print(f"mAP: {np.mean(map_list)}")
    cv_all.append(np.mean(map_list))

print("==================================")
print(f"mAP: {np.mean(cv_all)}")
print(f"mAP: {np.std(cv_all)}")
print("==================================")

['exp414', 'exp416', 'exp419', 'exp420', 'exp605', 'exp520', 'exp551']

==== cv 0 ===
AP Negative for Pneumonia: 0.8462930640824688
AP Typical Appearance: 0.8446479837886494
AP Indeterminate Appearance: 0.34127757810338244
AP Atypical Appearance: 0.4415757070034662
AP is_none: 0.8396885481792112
mAP: 0.6626965762314356

==== cv 1 ===
AP Negative for Pneumonia: 0.8149269088859116
AP Typical Appearance: 0.8701550446897675
AP Indeterminate Appearance: 0.37938749114413556
AP Atypical Appearance: 0.33445519230543747
AP is_none: 0.8035762250119015
mAP: 0.6405001724074307

==== cv 2 ===
AP Negative for Pneumonia: 0.8459289627593897
AP Typical Appearance: 0.8601550559273674
AP Indeterminate Appearance: 0.3217345537256371
AP Atypical Appearance: 0.3627717210424724
AP is_none: 0.8404145056934446
mAP: 0.6462009598296622

==== cv 3 ===
AP Negative for Pneumonia: 0.8255047464913853
AP Typical Appearance: 0.8485656613387746
AP Indeterminate Appearance: 0.3311800941830725
AP Atypical Appearance: 0.39

# all

In [21]:
exp = ["exp414", "exp415", "exp416", "exp417", "exp419", "exp420", "exp605", "exp520", "exp551", "exp552", "exp553"]

cv_all = []

print(exp)
for cv in range(5):
    pred_list = []
    pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
    pred_mean = np.mean(pred_list, axis=0)
    
    target = df[df.cv == cv][target_columns].values
    
    map_list = []
    print()
    print(f'==== cv {cv} ===')
    for i, col in enumerate(target_columns):
        map_ = average_precision_score(target[:, i], pred_mean[:, i])
        print(f"AP {col}: {map_}")
        map_list.append(map_)
    print(f"mAP: {np.mean(map_list)}")
    cv_all.append(np.mean(map_list))

print("==================================")
print(f"mAP: {np.mean(cv_all)}")
print(f"mAP: {np.std(cv_all)}")
print("==================================")

['exp414', 'exp415', 'exp416', 'exp417', 'exp419', 'exp420', 'exp605', 'exp520', 'exp551', 'exp552', 'exp553']

==== cv 0 ===
AP Negative for Pneumonia: 0.850952530078208
AP Typical Appearance: 0.8479670463762645
AP Indeterminate Appearance: 0.3342856331756847
AP Atypical Appearance: 0.4453915786677027
AP is_none: 0.8426027353628888
mAP: 0.6642399047321497

==== cv 1 ===
AP Negative for Pneumonia: 0.8168098353581016
AP Typical Appearance: 0.8704867210183729
AP Indeterminate Appearance: 0.37712118201240524
AP Atypical Appearance: 0.3392252837927331
AP is_none: 0.8056304952762023
mAP: 0.6418547034915631

==== cv 2 ===
AP Negative for Pneumonia: 0.8454152181739398
AP Typical Appearance: 0.8606352280270689
AP Indeterminate Appearance: 0.32227084092617475
AP Atypical Appearance: 0.36233560133127485
AP is_none: 0.8404925017692559
mAP: 0.6462298780455429

==== cv 3 ===
AP Negative for Pneumonia: 0.8279453804643679
AP Typical Appearance: 0.8493741025803431
AP Indeterminate Appearance: 0.331485

In [28]:
exp = ["exp414", "exp415", "exp416", "exp417", "exp419", "exp420", "exp605", "exp520", "exp551", "exp552", "exp553"]

for i in range(len(exp)):
    exp = ["exp414", "exp415", "exp416", "exp417", "exp419", "exp420", "exp605", "exp520", "exp551", "exp552", "exp553"]
    exp.pop(i)

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
        print()
        print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
            print(f"AP {col}: {map_}")
            map_list.append(map_)
        print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")

['exp415', 'exp416', 'exp417', 'exp419', 'exp420', 'exp605', 'exp520', 'exp551', 'exp552', 'exp553']

==== cv 0 ===
AP Negative for Pneumonia: 0.8511316522080687
AP Typical Appearance: 0.8473922910854397
AP Indeterminate Appearance: 0.3358828729591849
AP Atypical Appearance: 0.4400044214188208
AP is_none: 0.8432627174399584
mAP: 0.6635347910222944

==== cv 1 ===
AP Negative for Pneumonia: 0.8175656369733776
AP Typical Appearance: 0.870124794761812
AP Indeterminate Appearance: 0.37258567609343063
AP Atypical Appearance: 0.33897054309466373
AP is_none: 0.806234470052175
mAP: 0.6410962241950917

==== cv 2 ===
AP Negative for Pneumonia: 0.8430471386874773
AP Typical Appearance: 0.8600760775690728
AP Indeterminate Appearance: 0.320496544702923
AP Atypical Appearance: 0.3600465506987972
AP is_none: 0.8379489733217669
mAP: 0.6443230569960076

==== cv 3 ===
AP Negative for Pneumonia: 0.8280167381208331
AP Typical Appearance: 0.8489257566034031
AP Indeterminate Appearance: 0.3333394237216477
AP

In [29]:
exp = ["exp414", "exp415", "exp416", "exp417", "exp419", "exp605", "exp520", "exp551", "exp552", "exp553"]

for i in range(len(exp)):
    exp = ["exp414", "exp415", "exp416", "exp417", "exp419", "exp605", "exp520", "exp551", "exp552", "exp553"]
    exp.pop(i)

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
        print()
        print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
            print(f"AP {col}: {map_}")
            map_list.append(map_)
        print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")
    print()
    print()

['exp415', 'exp416', 'exp417', 'exp419', 'exp605', 'exp520', 'exp551', 'exp552', 'exp553']

==== cv 0 ===
AP Negative for Pneumonia: 0.8512552206599846
AP Typical Appearance: 0.8477635838345304
AP Indeterminate Appearance: 0.33682802004304824
AP Atypical Appearance: 0.44278160952368306
AP is_none: 0.8434041632412855
mAP: 0.6644065194605064

==== cv 1 ===
AP Negative for Pneumonia: 0.8180996896269439
AP Typical Appearance: 0.8707167271065773
AP Indeterminate Appearance: 0.3730468027520784
AP Atypical Appearance: 0.3463113067286401
AP is_none: 0.8072550666148175
mAP: 0.6430859185658114

==== cv 2 ===
AP Negative for Pneumonia: 0.8420847384915489
AP Typical Appearance: 0.8607252920741273
AP Indeterminate Appearance: 0.32061799961779097
AP Atypical Appearance: 0.36182518835288535
AP is_none: 0.8366177199919445
mAP: 0.6443741877056594

==== cv 3 ===
AP Negative for Pneumonia: 0.8298250813160805
AP Typical Appearance: 0.8491448446717473
AP Indeterminate Appearance: 0.33349876470433304
AP Aty

In [30]:
exp = ["exp414", "exp416", "exp417", "exp419", "exp605", "exp520", "exp551", "exp552", "exp553"]

for i in range(len(exp)):
    exp = ["exp414", "exp416", "exp417", "exp419", "exp605", "exp520", "exp551", "exp552", "exp553"]
    exp.pop(i)

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
        print()
        print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
            print(f"AP {col}: {map_}")
            map_list.append(map_)
        print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")
    print()
    print()

['exp416', 'exp417', 'exp419', 'exp605', 'exp520', 'exp551', 'exp552', 'exp553']

==== cv 0 ===
AP Negative for Pneumonia: 0.8486063410126826
AP Typical Appearance: 0.8470233949478659
AP Indeterminate Appearance: 0.33805707497311366
AP Atypical Appearance: 0.43918329775179055
AP is_none: 0.8406198595060564
mAP: 0.6626979936383018

==== cv 1 ===
AP Negative for Pneumonia: 0.8180214261013216
AP Typical Appearance: 0.8705786314629093
AP Indeterminate Appearance: 0.37475307179295847
AP Atypical Appearance: 0.34671472994361036
AP is_none: 0.8076988487238989
mAP: 0.6435533416049397

==== cv 2 ===
AP Negative for Pneumonia: 0.8418886890722018
AP Typical Appearance: 0.8607686654453222
AP Indeterminate Appearance: 0.3241083083336521
AP Atypical Appearance: 0.3651498832388068
AP is_none: 0.8364621180813017
mAP: 0.6456755328342569

==== cv 3 ===
AP Negative for Pneumonia: 0.8294468604156546
AP Typical Appearance: 0.8491863706889277
AP Indeterminate Appearance: 0.3341504610624837
AP Atypical Appea

In [31]:
exp = ["exp414", "exp416", "exp417", "exp605", "exp520", "exp551", "exp552", "exp553"]

for i in range(len(exp)):
    exp = ["exp414", "exp416", "exp417", "exp605", "exp520", "exp551", "exp552", "exp553"]
    exp.pop(i)

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
        print()
        print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
            print(f"AP {col}: {map_}")
            map_list.append(map_)
        print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")
    print()
    print()

['exp416', 'exp417', 'exp605', 'exp520', 'exp551', 'exp552', 'exp553']

==== cv 0 ===
AP Negative for Pneumonia: 0.8491738313958015
AP Typical Appearance: 0.8476122967031745
AP Indeterminate Appearance: 0.3360944428960307
AP Atypical Appearance: 0.4406063796126014
AP is_none: 0.8421460709104762
mAP: 0.6631266043036168

==== cv 1 ===
AP Negative for Pneumonia: 0.8193983333389508
AP Typical Appearance: 0.869631580846939
AP Indeterminate Appearance: 0.3758395057434572
AP Atypical Appearance: 0.34737375083160804
AP is_none: 0.807671037243853
mAP: 0.6439828416009616

==== cv 2 ===
AP Negative for Pneumonia: 0.842939077221215
AP Typical Appearance: 0.8607780431968866
AP Indeterminate Appearance: 0.3227279110518342
AP Atypical Appearance: 0.3619969550546304
AP is_none: 0.8383979392897405
mAP: 0.6453679851628614

==== cv 3 ===
AP Negative for Pneumonia: 0.8273415865053119
AP Typical Appearance: 0.8502595147823628
AP Indeterminate Appearance: 0.33588949489575265
AP Atypical Appearance: 0.399113

In [32]:
exp = ["exp414", "exp416", "exp417", "exp605", "exp520", "exp552", "exp553"]

for i in range(len(exp)):
    exp = ["exp414", "exp416", "exp417", "exp605", "exp520", "exp552", "exp553"]
    exp.pop(i)

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
        print()
        print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
            print(f"AP {col}: {map_}")
            map_list.append(map_)
        print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")
    print()
    print()

['exp416', 'exp417', 'exp605', 'exp520', 'exp552', 'exp553']

==== cv 0 ===
AP Negative for Pneumonia: 0.8481483595474083
AP Typical Appearance: 0.8489272446683543
AP Indeterminate Appearance: 0.33706923463945226
AP Atypical Appearance: 0.44356037848002644
AP is_none: 0.8415958082223847
mAP: 0.6638602051115253

==== cv 1 ===
AP Negative for Pneumonia: 0.8191166262970434
AP Typical Appearance: 0.8698800346199246
AP Indeterminate Appearance: 0.37366298589468117
AP Atypical Appearance: 0.3526734282944105
AP is_none: 0.8075142527087906
mAP: 0.64456946556297

==== cv 2 ===
AP Negative for Pneumonia: 0.8406446156016263
AP Typical Appearance: 0.860397708493787
AP Indeterminate Appearance: 0.3228736089377909
AP Atypical Appearance: 0.3614631092298516
AP is_none: 0.83675425453361
mAP: 0.6444266593593332

==== cv 3 ===
AP Negative for Pneumonia: 0.8280830019186595
AP Typical Appearance: 0.8504605498156826
AP Indeterminate Appearance: 0.33457871086587093
AP Atypical Appearance: 0.4010902077891283

In [33]:
    exp = ["exp414", "exp416", "exp417", "exp605", "exp520", "exp552", "exp553"]

    cv_all = []

    print(exp)
    for cv in range(5):
        pred_list = []
        pred_list.extend([np.load(f'/workspace/output/oof/{exp_name}_cv{cv}.npy') for exp_name in exp])
        pred_mean = np.mean(pred_list, axis=0)

        target = df[df.cv == cv][target_columns].values

        map_list = []
        print()
        print(f'==== cv {cv} ===')
        for i, col in enumerate(target_columns):
            map_ = average_precision_score(target[:, i], pred_mean[:, i])
            print(f"AP {col}: {map_}")
            map_list.append(map_)
        print(f"mAP: {np.mean(map_list)}")
        cv_all.append(np.mean(map_list))

    print("==================================")
    print(f"mAP: {np.mean(cv_all)}")
    print(f"mAP: {np.std(cv_all)}")
    print("==================================")
    print()
    print()

['exp414', 'exp416', 'exp417', 'exp605', 'exp520', 'exp552', 'exp553']

==== cv 0 ===
AP Negative for Pneumonia: 0.8485955508799609
AP Typical Appearance: 0.8492407874982908
AP Indeterminate Appearance: 0.33468042776602736
AP Atypical Appearance: 0.4482565636877983
AP is_none: 0.8411446789233791
mAP: 0.6643836017510913

==== cv 1 ===
AP Negative for Pneumonia: 0.817944587490543
AP Typical Appearance: 0.8700706385654374
AP Indeterminate Appearance: 0.3759978892080396
AP Atypical Appearance: 0.348763676480658
AP is_none: 0.8067769784043641
mAP: 0.6439107540298086

==== cv 2 ===
AP Negative for Pneumonia: 0.8445618068202485
AP Typical Appearance: 0.8606073726153227
AP Indeterminate Appearance: 0.3241582374078378
AP Atypical Appearance: 0.3643069942615251
AP is_none: 0.8398047454365162
mAP: 0.64668783130829

==== cv 3 ===
AP Negative for Pneumonia: 0.8287790063618565
AP Typical Appearance: 0.8507379548148515
AP Indeterminate Appearance: 0.33401513240823055
AP Atypical Appearance: 0.4036382